In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# --- Load MNIST dataset ---

transform = transforms.Compose([
    transforms.ToTensor(),                                                                          # Convert image to tensor
    transforms.Normalize((0.1307,), (0.3081,))                                                      # Normalize with mean/std of MNIST
])

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

# --- Define a linear classifier model ---

class LinearClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(28*28, 10)                                                             # 784 → 10

    def forward(self, x):
        x = x.view(-1, 28*28)                                                                      # flatten each image
        return self.fc(x)

model = LinearClassifier()

# --- Loss and optimizer ---

criterion = nn.CrossEntropyLoss()                                                                  # good for classification
optimizer = optim.SGD(model.parameters(), lr=0.1)

# --- Training loop ---

for epoch in range(5):                                                                             # 5 epochs
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# --- Evaluation ---

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
